# Voorbereiding

In [127]:
# zorgen voor de juiste modules
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV, cross_validate, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, BayesianRidge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA

from xgboost import XGBRegressor
from pandas import DataFrame

# instellingen voor panda weergave aanpassen
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",None)    # alle rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen

## Dataframe parameters

In [128]:
# locatie van dataset 
DF_LOCATION = 'C:/Users/LuukJ/Desktop/DEP/Execute/Project/WMO Casus/Data/df_dataset_WMO.parquet.gzip'

In [129]:
# manier van laden dataset. Bijvoorbeeld read_parquet of read_csv
DF_READ = pd.read_parquet

## X & Y parameters

In [130]:
# de kolommen die uit de X dataset moeten worden gehaald. Dat is in ieder geval de y en eventueel nog meer kolommen.
X_DROP_VALUES = ['wmoclientenper1000inwoners','wmoclienten']

In [131]:
# de kolom die wordt gebruikt als y value
Y_VALUE = ['wmoclientenper1000inwoners']

In [132]:
# test size voor de train/test split
TEST_SIZE = 0.3

In [133]:
# random state voor de train/test split. Bijvoorbeeld random_state = 42 als vaste seed voor reproduceerbaarheid
RANDOM_STATE = None

## Pipeline parameters

In [134]:
# strategy en waarde om te vullen bij lege categorische kolommen
NAN_VALUES_CAT_STRATEGY = 'constant'
NAN_VALUES_CAT_VALUES = 'Missing'

In [135]:
# waarden om in te vullen bij lege numerieke kolommen. Bijvoorbeeld mean of median
NAN_VALUES_NUM_STRATEGY = 'mean'

## Model parameters

In [136]:
# manier van cross validate in de modellen. Bijvoorbeeld 10 of RepeatedKFold(n_splits=30, n_repeats=5, random_state=1)
CROSS_VALIDATE = 10

In [137]:
# manier van scoren in de modellen
MODEL_SCORING = 'neg_mean_squared_error'

## Scoring parameters

later toevoegen als we meerdere manieren van scoren hebben. Dus niet alleen maar de RSMLE

# Dataframe, X en y prepareren

In [139]:
df = DF_READ(DF_LOCATION)

In [140]:
# droppen van de rijen waar de y_value leeg is, anders kunnen de modellen er niet mee overweg
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=Y_VALUE,
    inplace=True
)

In [141]:
# van object kolommen category maken 
df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').astype('category')

In [142]:
# X en y aanmaken
X = df.drop(X_DROP_VALUES, axis=1)
y = df[Y_VALUE]

In [143]:
# splitsen van X en y in train/test. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)

In [144]:
# splitsen van X_train in categorische en numerieke kolommen, om apart te kunnen transformeren
cat_cols = X_train.select_dtypes(include=['category']).columns
num_cols = X_train.select_dtypes(include=['int64','float64']).columns

# Pipelines

In [145]:
# pipelines (pl) maken voor imputing, scaling en OneHotEncoding per datatype 

# categorie met waarde die is gegeven aan "MISSING" toevoegen
for col in cat_cols:
    # need to add category for missings, otherwise error with OneHotEncoding (volgens mij ook met alleen imputing)
    X_train[col].cat.add_categories(NAN_VALUES_CAT_VALUES, inplace=True)
categories = [X_train[col].cat.categories for col in cat_cols]

# pipeline voor categorial datatype
pl_ppc_cat = make_pipeline(
     SimpleImputer(
         missing_values = np.nan
        ,strategy = NAN_VALUES_CAT_STRATEGY
        ,fill_value = NAN_VALUES_CAT_VALUES)
    ,OneHotEncoder(categories=categories)
)

# pipeline voor numeriek datatype
pl_ppc_num = make_pipeline(
     SimpleImputer(
         missing_values = np.nan
        ,strategy = NAN_VALUES_NUM_STRATEGY)
    ,StandardScaler()
    ,PCA() # PCA heeft behoorlijk wat (positieve) invloed op de scores
)

In [146]:
# pipelines maken om de preprocessing van de imputing te combineren
pl_ppc_total = make_column_transformer(
     (pl_ppc_cat, cat_cols)
    ,(pl_ppc_num, num_cols)
    ,remainder = 'drop'
)

In [147]:
# pipeline maken voor LinearRegression 
pl_lr = make_pipeline(
     pl_ppc_total
    ,LinearRegression()
)

In [148]:
# pipeline maken voor RidgeRegression 
pl_rr = make_pipeline(
     pl_ppc_total
    ,Ridge()
)

In [149]:
# pipeline maken voor Lasso
pl_lasso = make_pipeline(
      pl_ppc_total
     ,Lasso(alpha=0.001)
)

In [150]:
# pipeline maken voor KNN
pl_knn = make_pipeline(
      pl_ppc_total
     ,KNeighborsRegressor()
)

In [151]:
# pipeline maken voor SVM
pl_svm = make_pipeline(
      pl_ppc_total
     ,SVR()
)

In [152]:
# pipeline maken voor XGB
pl_xgb = make_pipeline(
      pl_ppc_total
     ,XGBRegressor()
) 

# Modellen

### Lineair Regression

In [153]:
# scores voor LR berekenen
lr_scores = cross_validate(
    pl_lr, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring=([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

### Ridge Regression

In [154]:
rr_scores = cross_validate(
    pl_rr, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

### Lasso

In [155]:
lasso_scores = cross_validate(
    pl_lasso, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

### K Nearest Neighbor

In [156]:
knn_scores = cross_validate(
    pl_knn, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

### Support Vector Machines

In [157]:
svm_scores = cross_validate(
    pl_svm, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

C:\Users\LuukJ\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\LuukJ\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\LuukJ\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\LuukJ\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

### XGBoost

In [158]:
xgb_scores = cross_validate(
    pl_xgb, X_train, y_train,
    cv = CROSS_VALIDATE,
    scoring = ([MODEL_SCORING]),
    return_train_score=True,
    return_estimator=True,
)

## Scores per model

In [159]:
# functie maken om op basis van de cv scores, het beste RMLSE model te selecteren 
def get_best_model_rmsle(cv_scores):
    """
    Return best (most conservative) model from cross_validate object.
    
    Uses np.argmax to find bottomright point == largest RMLSE
    """
    index = np.argmax(np.sqrt(-cv_scores['train_neg_mean_squared_error']))
    model = cv_scores['estimator'][index]
    rmsle = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
    return (rmsle)

In [160]:
# toevoegen van de beste scores, per model, aan een lijst
scores = []
scores.append(('Linear Regression',  get_best_model_rmsle(lr_scores)))
scores.append(('Ridge Regression',  get_best_model_rmsle(rr_scores)))
scores.append(('Lasso', get_best_model_rmsle(lasso_scores)))
scores.append(('K Nearest Neighbor', get_best_model_rmsle(knn_scores)))
scores.append(('Support Vector Machines', get_best_model_rmsle(svm_scores)))
scores.append(('XGBoost', get_best_model_rmsle(xgb_scores)))

In [161]:
# maken van een dataframe met daarin de gesorteerde scores per model
scores = pd.DataFrame(scores)
scores.columns = ['Algorithm', 'RMSLE'] 
scores['RMSLE'] = scores['RMSLE'].map('{:,.4f}'.format)
scores = scores.sort_values('RMSLE', ascending=True)
scores = scores.reset_index(drop=True)
scores

,Algorithm,RMSLE
0,XGBoost,17.3355
1,Ridge Regression,17.4656
2,K Nearest Neighbor,17.6550
3,Lasso,18.0756
4,Linear Regression,18.2156
5,Support Vector Machines,23.8914


In [162]:
# and the winner is...
print(f"Het algoritme met de laagste RMSLE is:\n\n{scores.iloc[0,0]}, met een RMSLE van {scores.iloc[0,1]}")

Het algoritme met de laagste RMSLE is:

XGBoost, met een RMSLE van 17.3355
